In [ ]:
import arcpy
import arcgis
import pandas as pd
import os

In [ ]:
workspace_folder = r"C:\data\Geonames"
geonames_file_path = os.path.join(workspace_folder, "cities500.csv")

In [ ]:
geonames_column_names = [
    "geonameid",
    "name",
    "ascii_name",
    "alternate_names",
    "latitude",
    "longitude",
    "feature_class",
    "feature_code",
    "country_code",
    "cc2",
    "admin1_code",
    "admin2_code",
    "admin3_code",
    "admin4_code",
    "population",
    "elevation",
    "dem",
    "timezone",
    "modification_date"
]
geonames_dtypes_dict = {
    0: int, # geonameid
    1: str,  # name
    2: str,  # asciiname
    3: str,  # alternatenames
    4: float, # latitude
    5: float, # longitude
    6: str, # feature class
    7: str, # feature code
    8: str, # country code
    9: str, # cc2
    10: str, # admin1 code
    11: str, # admin2 code
    12: str, # admin3 code
    13: str, # admin4 code
    14: int, # population
    15: float, # elevation
    16: int, # dem (digital elevation model)
    17: str, # timezone
    18: str # modification date yyyy-MM-dd
}

In [ ]:
cities500_df = pd.read_csv(geonames_file_path, sep='\t', header=None, names=geonames_column_names, dtype=geonames_dtypes_dict)
cities500_df

In [ ]:
def buffer_from_xy(df, x_column, y_column, out_fclass, distance_linear_unit):
    """
    Creates a geodesic buffer using a Pandas dataframe representing x,y locations.
    Returns an in-memory feature class.
    """
    arcpy.env.overwriteOutput = True
    spatial_df = arcgis.features.GeoAccessor.from_xy(df, x_column=x_column, y_column=y_column)
    input_fc = spatial_df.spatial.to_featureclass("in_memory/input_features")
    gp_result = arcpy.analysis.Buffer(input_fc, out_fclass, distance_linear_unit, method="GEODESIC")
    print(gp_result.getMessages())
    if 4 != gp_result.status:
        raise Exception("GP Tool failed!")
    
    buffers_fc = gp_result[0]
    return buffers_fc

In [ ]:
max_rows = 10000

In [ ]:
buffers_fc = buffer_from_xy(cities500_df[:max_rows], "longitude", "latitude", "in_memory/buffers", "45 kilometers")
buffers_fc

In [ ]:
buffers_sdf = arcgis.features.GeoAccessor.from_featureclass(buffers_fc)
buffers_sdf

In [ ]:
columns = ["name", "SHAPE"]

In [ ]:
gis = arcgis.gis.GIS("pro")
focus_map = gis.map("Europe")
buffers_sdf[columns].spatial.plot(focus_map)
focus_map